# Short-Term Memory Experiments

This notebook explores short-term memory functionality and optimization for AI agents.

In [ ]:
import sys
from pathlib import Path
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd

# Add backend to path
sys.path.insert(0, str(Path.cwd().parent.parent / "backend"))

from src.job_automation.core.memory import ShortTermMemory, MemoryManager

print("Short-term memory environment ready!")

## Basic Short-Term Memory Usage

In [ ]:
# Create short-term memory instance
stm = ShortTermMemory(max_size=10)

# Add some sample interactions
sample_interactions = [
    ("Hello", "Hi there! How can I help you today?"),
    ("I'm looking for software engineering jobs", "Great! What type of software engineering role interests you?"),
    ("Full-stack development with Python and React", "Excellent! I'll help you find full-stack positions with Python and React."),
    ("What companies should I target?", "Based on your preferences, I recommend looking at tech companies that use modern web stacks."),
    ("Can you help me write a cover letter?", "Absolutely! I can help you craft a personalized cover letter.")
]

for user_input, agent_response in sample_interactions:
    stm.add_interaction(user_input, agent_response, {"conversation_id": "test_001"})

print(f"Added {len(sample_interactions)} interactions to memory")
print(f"Memory contains {len(stm.memory)} interactions")

In [ ]:
# Retrieve and display recent interactions
recent = stm.get_recent_interactions(3)
print("Recent interactions:")
for i, interaction in enumerate(recent, 1):
    print(f"\n{i}. User: {interaction['user_input']}")
    print(f"   Agent: {interaction['agent_response']}")
    print(f"   Time: {interaction['timestamp']}")

## Context Management Experiments

In [ ]:
# Set context variables
stm.set_context("user_preferences", {
    "job_type": "full-stack",
    "technologies": ["Python", "React", "PostgreSQL"],
    "experience_level": "mid-level",
    "remote_preference": "hybrid"
})

stm.set_context("current_session", {
    "session_id": "sess_20250119_001",
    "start_time": datetime.now().isoformat(),
    "user_id": "user_123"
})

# Retrieve context
preferences = stm.get_context("user_preferences")
session = stm.get_context("current_session")

print("User preferences:")
print(json.dumps(preferences, indent=2))
print("\nSession info:")
print(json.dumps(session, indent=2))

## Memory Size and Performance Testing

In [ ]:
# Test memory size limits
test_memory = ShortTermMemory(max_size=5)

# Add more interactions than the limit
for i in range(10):
    test_memory.add_interaction(
        f"Test input {i}",
        f"Test response {i}",
        {"test_batch": "size_limit_test", "index": i}
    )

print(f"Added 10 interactions, memory contains: {len(test_memory.memory)}")
print("\nStored interactions:")
for interaction in test_memory.memory:
    print(f"- {interaction['user_input']} (index: {interaction['metadata']['index']})")

In [ ]:
# Performance testing - adding many interactions
import time

perf_memory = ShortTermMemory(max_size=1000)
num_interactions = 500

start_time = time.time()

for i in range(num_interactions):
    perf_memory.add_interaction(
        f"Performance test input {i}",
        f"Performance test response {i}",
        {"batch": "performance_test", "index": i}
    )

end_time = time.time()
duration = end_time - start_time

print(f"Added {num_interactions} interactions in {duration:.4f} seconds")
print(f"Average time per interaction: {(duration/num_interactions)*1000:.2f} ms")
print(f"Memory size: {len(perf_memory.memory)} interactions")

## Memory Export and Analysis

In [ ]:
# Export memory to analyze patterns
memory_data = stm.to_dict()

# Analyze interaction patterns
interactions_df = pd.DataFrame(memory_data['memory'])

if not interactions_df.empty:
    # Convert timestamp to datetime
    interactions_df['timestamp'] = pd.to_datetime(interactions_df['timestamp'])
    
    # Analyze interaction lengths
    interactions_df['user_input_length'] = interactions_df['user_input'].str.len()
    interactions_df['agent_response_length'] = interactions_df['agent_response'].str.len()
    
    print("Interaction analysis:")
    print(f"Average user input length: {interactions_df['user_input_length'].mean():.1f} characters")
    print(f"Average agent response length: {interactions_df['agent_response_length'].mean():.1f} characters")
    
    # Plot interaction lengths
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.hist(interactions_df['user_input_length'], bins=10, alpha=0.7)
    plt.title('User Input Length Distribution')
    plt.xlabel('Length (characters)')
    plt.ylabel('Frequency')
    
    plt.subplot(1, 2, 2)
    plt.hist(interactions_df['agent_response_length'], bins=10, alpha=0.7)
    plt.title('Agent Response Length Distribution')
    plt.xlabel('Length (characters)')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No interactions to analyze")

## Integration with Memory Manager

In [ ]:
# Test memory manager integration
manager = MemoryManager(short_term_max_size=20)

# Add some interactions through the manager
job_search_conversation = [
    ("I need help finding a job", "I'd be happy to help you with your job search!"),
    ("I'm a Python developer", "Great! Python is in high demand. What level of experience do you have?"),
    ("5 years of experience", "Excellent! With 5 years of Python experience, you're well-positioned for senior roles."),
    ("I prefer remote work", "I'll focus on remote Python opportunities for you.")
]

for user_input, agent_response in job_search_conversation:
    manager.add_interaction(user_input, agent_response, {
        "topic": "job_search",
        "user_profile": "python_developer_5yr"
    })

# Set relevant context
manager.set_context("user_profile", {
    "skills": ["Python", "Django", "FastAPI", "PostgreSQL"],
    "experience_years": 5,
    "work_preference": "remote"
})

# Get memory statistics
stats = manager.get_memory_stats()
print("Memory Manager Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value}")

# Get recent interactions
recent_interactions = manager.get_recent_interactions(2)
print("\nRecent interactions:")
for i, interaction in enumerate(recent_interactions, 1):
    print(f"\n{i}. User: {interaction['user_input']}")
    print(f"   Agent: {interaction['agent_response']}")
    print(f"   Topic: {interaction['metadata']['topic']}")

## Save Experiment Results

In [ ]:
# Save experimental data
experiment_results = {
    "experiment_date": datetime.now().isoformat(),
    "memory_performance": {
        "interactions_processed": num_interactions,
        "total_time": duration,
        "avg_time_per_interaction": (duration/num_interactions)*1000
    },
    "memory_stats": stats,
    "sample_interactions": len(sample_interactions)
}

# Save to file
output_dir = Path.cwd().parent / "experiments"
output_dir.mkdir(exist_ok=True)

with open(output_dir / "short_term_memory_experiment.json", "w") as f:
    json.dump(experiment_results, f, indent=2)

print(f"Experiment results saved to {output_dir / 'short_term_memory_experiment.json'}")